Cetta Maulana Andhika
1103213119
TK-45-04
dataset : credit_score.csv

### Import semua library

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import accuracy_score, classification_report,mean_absolute_error,mean_absolute_percentage_error,mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb



### Baca Dataset

In [2]:
data = pd.read_csv("datatset\\train.csv")

data.info()


C:\Users\cetta\AppData\Local\Temp\ipykernel_18948\3591381227.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("datatset\\train.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [3]:
#Drop Meta
data = data.drop(columns=['ID',"Customer_ID","SSN","Credit_History_Age",'Name',"Type_of_Loan"]) 
data = data.dropna(axis=0) #Drop missing data row
data = data.applymap(lambda x: x.replace('_', '') if isinstance(x, str) else x) #dremove "_" in data

#Convert categorical columns into categorical numerical
categorical = ['Month',"Occupation","Credit_Mix","Payment_of_Min_Amount","Payment_Behaviour","Credit_Score"]
label_encoder = LabelEncoder()
for x in categorical:
    data[x] = label_encoder.fit_transform(data[x])

# Filter out negative values in numeric columns
data = data.apply(pd.to_numeric,errors='coerce')
data = data.applymap(lambda x: x if x >= 0 else None)
data.dropna(axis=0,inplace=True)

# Calculate quartiles for each feature
quartiles = data.quantile([0.25, 0.75])

# Calculate IQR (Interquartile Range) for each feature
IQR = quartiles.loc[0.75] - quartiles.loc[0.25]

# Define lower and upper bounds for each feature
lower_bound = (quartiles.loc[0.25] - 1.5 * IQR).to_dict()
upper_bound = (quartiles.loc[0.75] + 1.5 * IQR).to_dict()

# Filter out data points outside the bounds for each feature
filtered_data = data.copy()
for feature in data.columns:
    lower_bound_value = lower_bound[feature]
    upper_bound_value = upper_bound[feature]
    filtered_data = filtered_data[(filtered_data[feature] >= lower_bound_value) & (filtered_data[feature] <= upper_bound_value)]
data = filtered_data



In [4]:
# Separate features (X) and target variable (y)
X = data.drop('Credit_Score', axis=1)  # Replace 'target_variable_name' with the name of your target variable
y = data['Credit_Score']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=10)  # You can adjust the number of neighbors

# Train the KNN classifier
knn_classifier.fit(X_train_scaled, y_train)

# Predict the labels for test set
y_pred = knn_classifier.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
mean_error = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mean_error)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

Accuracy: 0.7089895808093046

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.57      0.56       674
           1       0.71      0.67      0.69      1185
           2       0.75      0.77      0.76      2268

    accuracy                           0.71      4127
   macro avg       0.67      0.67      0.67      4127
weighted avg       0.71      0.71      0.71      4127

Mean Absolute Error: 0.4099830385267749
Mean Absolute Percentage Error (MAPE): 613283981241148.2
Mean Squared Error (MSE): 0.6479282771989339


In [5]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the number of estimators

# Train the Random Forest classifier
rf_classifier.fit(X_train, y_train)

# Predict the labels for test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
mean_error = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mean_error)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

Accuracy: 0.7872546644051369

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.69      0.71      1323
           1       0.77      0.79      0.78      2353
           2       0.81      0.81      0.81      4578

    accuracy                           0.79      8254
   macro avg       0.77      0.76      0.77      8254
weighted avg       0.79      0.79      0.79      8254

Mean Absolute Error: 0.2984007753816331
Mean Absolute Percentage Error (MAPE): 437592306899823.0
Mean Squared Error (MSE): 0.46971165495517325


In [6]:
# Initialize XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', random_state=42)  

# Train the XGBoost classifier
xgb_classifier.fit(X_train, y_train)

# Predict the labels for test set
y_pred = xgb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
mean_error = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mean_error)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

SyntaxError: invalid syntax (1669620205.py, line 21)